In [1]:
from anet_api.db import Team, Athlete, Meet, TeamCreate
from anet_api.db.utils import create_item
import requests as r
from sqlmodel import select, Session
from anet_api.db.database import SessionLocal
import json

In [2]:
api_url = 'http://127.0.0.1:8000/'

In [3]:
metro_teams = [
    dict(anet_id=489, name='Ballard'),
    dict(anet_id=415, name='Bishop Blanchet'),
    dict(anet_id=416, name='Chief Sealth'),
    dict(anet_id=417, name='Cleveland'),
    dict(anet_id=418, name='Eastside Catholic'),
    dict(anet_id=492, name='Franklin'),
    dict(anet_id=483, name='Garfield'),
    dict(anet_id=419, name='Holy Names Academy'),
    dict(anet_id=420, name='Ingraham'),
    dict(anet_id=421, name='Lakeside'),
    dict(anet_id=78877, name='Lincoln'),
    dict(anet_id=422, name='Nathan Hale'),
    dict(anet_id=423, name='O\'Dea'),
    dict(anet_id=424, name='Rainier Beach'),
    dict(anet_id=498, name='Roosevelt'),
    dict(anet_id=425, name='Seattle Prep'),
    dict(anet_id=426, name='West Seattle'),
    dict(anet_id=309,name='Seattle Academy'),
    dict(anet_id=414,name='Bainbridge')
]

In [4]:
for team in metro_teams:
    r.post(api_url + 'db/addTeam',data=json.dumps(team))

In [5]:
def get_all_teams(session: Session):
    stmt = select(Team)
    return session().exec(stmt).all()

def insert_meets_and_athletes(params):
    q = r.get(api_url + 'anet/getTeam', params = params)
    meets = q.json()['schedule']
    roster = q.json()['roster']
    for meet in meets:
        response = r.post(api_url + 'db/addMeet',data=json.dumps(meet))
        if response.status_code != 200:
            print(meet['meet'], response.text)
    for athlete in roster:
        response = r.post(api_url + 'db/addAthlete', data=json.dumps(athlete))
        if response.status_code != 200:
            print(athlete['anet_id'], response.text)
    return

In [6]:
season = 2019
sport = 'xc'
for team in get_all_teams(SessionLocal):
    print(team)
    params = dict(team_id = team.anet_id, season = season, sport = sport)
    insert_meets_and_athletes(params)

name='Test High School' created_at=datetime.datetime(2024, 12, 19, 8, 1, 24, 443894) last_edited=datetime.datetime(2024, 12, 19, 0, 3, 45, 967000) anet_id=1 id=1
name='Garfield' created_at=datetime.datetime(2024, 12, 19, 8, 1, 24, 443894) last_edited=datetime.datetime(2024, 12, 19, 8, 7, 32, 184226) anet_id=493 id=2
Tahoma Coed Relays (Jamboree) {"detail":"Meet already exists"}
ING/Franklin/Garfield/Ballard/Bainbridge {"detail":"Meet already exists"}
Brooks Fort Steilacoom Invitational {"detail":"Meet already exists"}
Garfield/WS/RB/Clev/CS/Bainbridge/Roosevelt {"detail":"Meet already exists"}
Bellevue Cross Country Invite {"detail":"Meet already exists"}
Bainbridge / Franklin / Garfield / Cleveland @ Battle Point Park {"detail":"Meet already exists"}
NIKE HOLE IN THE WALL XC INVITATIONAL {"detail":"Meet already exists"}
Hale/Garfield/Franklin/Ing/Ballard/Roos/ WS/CS/Clev/RB/Lincoln {"detail":"Meet already exists"}
Metro League Championships {"detail":"Meet already exists"}
3A Seaking 

In [7]:
from sqlalchemy import extract
def get_all_meets_by_season(session: Session, season: int):
    stmt = select(Meet).where(extract('year', Meet.date) == season)
    return session.exec(stmt).all()

In [8]:

meets = get_all_meets_by_season(SessionLocal(),2023)




In [9]:
meets

[Meet(anet_id=224755, last_edited=datetime.datetime(2024, 12, 19, 8, 7, 43, 493253), venue='Myrtle Point High School', city='Myrtle Point', zipcode='97458', id=1, created_at=datetime.datetime(2024, 12, 19, 8, 1, 24, 443894), meet='Myrtle Point Harvest Festival', address='717 4th Street', state='OR', date=datetime.date(2023, 9, 9)),
 Meet(anet_id=223847, last_edited=datetime.datetime(2024, 12, 19, 8, 7, 43, 508351), venue='Colver Sports Fields', city='Talent', zipcode='97540', id=2, created_at=datetime.datetime(2024, 12, 19, 8, 1, 24, 443894), meet='Rogue XC Invitational ', address='6450 Colver Rd', state='OR', date=datetime.date(2023, 9, 16)),
 Meet(anet_id=222689, last_edited=datetime.datetime(2024, 12, 19, 8, 7, 43, 516039), venue='Glide High School', city='Glide', zipcode='97443', id=3, created_at=datetime.datetime(2024, 12, 19, 8, 1, 24, 443894), meet="Glide Invitational - This Run's For Maynard", address=None, state='OR', date=datetime.date(2023, 9, 21)),
 Meet(anet_id=228583, las

In [7]:
def insert_new_result(meet_id: int):
    params = dict(sport = 'xc', meet_id = meet_id)
    s = r.get(api_url + 'anet/getResults',params=params)

    all_results = []

    for race in s.json()['races']:
        rdb = r.post(api_url + 'db/addRace',data=race)
        all_results += race['results']

    for result in all_results:
        response = r.post(api_url + 'db/addResult', data = json.dumps(result))
        if response.status_code == 422:
            print(response.text)
    return "All done!"

In [10]:
params = dict(sport = 'xc', meet_id = 161636)
s = r.get(api_url + 'anet/getResults',params=params)
s.json()['races']

[{'race_details': {'anet_race_id': 675619,
   'gender': 'M',
   'race_name': '2 Miles Varsity',
   'division': 'Varsity',
   'place_depth': 7,
   'score_depth': 5,
   'start_time': None,
   'distance': 3218},
  'results': [{'anet_id': 29180622,
    'anet_meet_id': 161636,
    'anet_athlete_id': 10462477,
    'result': '10:39',
    'place': 0,
    'pb': True,
    'sb': True,
    'distance': None,
    'anet_team_id': 306,
    'first_name': 'Keegan',
    'last_name': 'Minahan',
    'team': 'The Northwest School',
    'grade': None},
   {'anet_id': 29180619,
    'anet_meet_id': 161636,
    'anet_athlete_id': 10462463,
    'result': '10:56',
    'place': 0,
    'pb': True,
    'sb': True,
    'distance': None,
    'anet_team_id': 306,
    'first_name': 'Milo',
    'last_name': 'Greenberg',
    'team': 'The Northwest School',
    'grade': None},
   {'anet_id': 29180617,
    'anet_meet_id': 161636,
    'anet_athlete_id': 14017634,
    'result': '11:09',
    'place': 0,
    'pb': True,
    'sb

In [8]:
for meet in meets:
    insert_new_result(meet.anet_id)

NameError: name 'meets' is not defined

In [26]:
session = SessionLocal()
team = Team(anet_id=111, name="New Team")
athlete = Athlete(anet_id=222, first_name="John", last_name="Doe", gender="F")
meet = Meet(anet_id=333, name="New Meet", venue="Place", date="2023-01-01")

session.add(team)
session.add(athlete)
session.add(meet)
session.commit()

In [11]:
import requests as r
url = 'https://www.athletic.net/api/v1/Event/Events'
data = dict(
        start='2022-11-01',
        end='2022-11-30',
        levelMask=4,
        sportMask=2,
        state='WA',
        country='us',
    )

res = r.post(url,json=data)

In [12]:
res.json()

{'events': [{'IDMeet': 224294,
   'Sport': 'XC',
   'MeetName': 'Forks HS League Meet (CANCELED)',
   'SeasonId': 2023,
   'StartDate': '0001-01-01T00:00:00',
   'EndDate': '2023-09-19T00:00:00',
   'Virtual': False,
   'LocationName': 'Olympic Natural Resources Center',
   'StreetAddress': '1455 S Forks Ave',
   'City': 'Forks',
   'PostalCode': '98331',
   'State': 'WA',
   'OwnerID': 364,
   'CalendarLock': 0,
   'UCalendarLock': 0,
   'RegEnd': None,
   'HasResults': 0,
   'LevelMask': 6,
   'HostName': 'Forks',
   'MascotUrl': '//lh3.googleusercontent.com/AUH2hZZ7KFJd1HnvbCxwUWO5zaQuN2YUpzeckUG9V9bd6Y6BpCO4ajPePXdkY0zjbVNaEfaKkhQsHPFr08_kMw',
   'Lat': 47.9330028,
   'Long': -124.3944981,
   'score': 0.0,
   'CalCount': 1,
   'OffDays': '{}',
   'Data': None,
   'rsUrl': None,
   'LiveID': None,
   'LivePublished': True,
   'VideoURL': None,
   'Website': '',
   'Country': 'US'},
  {'IDMeet': 225599,
   'Sport': 'XC',
   'MeetName': 'Removed',
   'SeasonId': 2023,
   'StartDate': 